In [ ]:
# Célula 1: Configurações e Importações
import requests
import pandas as pd
import os

CNPJ_EMPRESA = "03045711000170"
ANOS = [2023, 2024, 2025]
ORDENACAO = 4 #ordenação por data de pagamento
API_KEY = "45390d10a2fcb638bdaa4f34f27805f1"

FASENOMES = {
    1: "Empenhos",
    2: "Liquidações",
    3: "Pagamentos"
}


In [ ]:
# Célula 2: Função para coletar os dados
def coletar_dados_por_fase(fase, ano):
    pagina = 1
    resultados = []

    while True:
        url = "https://api.portaldatransparencia.gov.br/api-de-dados/despesas/documentos-por-favorecido"
        params = {
            "codigoPessoa": CNPJ_EMPRESA,
            "fase": fase,
            "ano": ano,
            "pagina": pagina,
            "ordenacaoResultado": ORDENACAO
        }
        headers = {
            "chave-api-dados": API_KEY,
            "accept": "*/*"
        }
        
        # Adicione um print para ver a URL completa que está sendo chamada
        # print(f"Chamando URL: {requests.Request('GET', url, params=params).prepare().url}")

        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Erro na fase {fase}, ano {ano}, página {pagina}: status {response.status_code}, resposta: {response.text}")
            break
        dados = response.json()
        if not dados:
            break
        resultados.extend(dados)
        print(f"✅ {FASENOMES[fase]} {ano}: Página {pagina} com {len(dados)} registros")
        pagina += 1
    return pd.DataFrame(resultados)

# Você pode testar a função aqui com uma chamada única antes de rodar o loop completo
# df_teste = coletar_dados_por_fase(1, 2024)
# print("Primeiros 5 registros do teste:")
# print(df_teste.head())# Célula 2: Função para coletar os dados
def coletar_dados_por_fase(fase, ano):
    pagina = 1
    resultados = []

    while True:
        url = "https://api.portaldatransparencia.gov.br/api-de-dados/despesas/documentos-por-favorecido"
        params = {
            "codigoPessoa": CNPJ_EMPRESA,
            "fase": fase,
            "ano": ano,
            "pagina": pagina,
            "ordenacaoResultado": ORDENACAO
        }
        headers = {
            "chave-api-dados": API_KEY,
            "accept": "*/*"
        }
        
        # Adicione um print para ver a URL completa que está sendo chamada
        # print(f"Chamando URL: {requests.Request('GET', url, params=params).prepare().url}")

        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"⚠️ Erro na fase {fase}, ano {ano}, página {pagina}: status {response.status_code}, resposta: {response.text}")
            break
        dados = response.json()
        if not dados:
            break
        resultados.extend(dados)
        print(f"✅ {FASENOMES[fase]} {ano}: Página {pagina} com {len(dados)} registros")
        pagina += 1
    return pd.DataFrame(resultados)

# Você pode testar a função aqui com uma chamada única antes de rodar o loop completo
#df_teste = coletar_dados_por_fase(1, 2024)
#print("Primeiros 5 registros do teste:")
#print(df_teste.head())

✅ Empenhos 2024: Página 1 com 15 registros
✅ Empenhos 2024: Página 2 com 15 registros
✅ Empenhos 2024: Página 3 com 15 registros
✅ Empenhos 2024: Página 4 com 15 registros
✅ Empenhos 2024: Página 5 com 15 registros
✅ Empenhos 2024: Página 6 com 15 registros
✅ Empenhos 2024: Página 7 com 3 registros
Primeiros 5 registros do teste:
         data                documento documentoResumido  \
0  31/12/2024  153010152442024NE001263      2024NE001263   
1  31/12/2024  344001342012022NE000223      2022NE000223   
2  31/12/2024  153010152442024NE000060      2024NE000060   
3  31/12/2024  153010152442022NE000187      2022NE000187   
4  31/12/2024  153162152362022NE000042      2022NE000042   

                                          observacao         funcao  \
0                MANUTENCAO PREDIAL - MARIA DA GRACA  12 - Educação   
1  REGISTRO DE CANCELAMENTO DE RESTOS A PAGAR BLO...   13 - Cultura   
2  REGISTRO DE ANULACAO/REFORCO/CANCELAMENTO DO E...  12 - Educação   
3  REGISTRO DE CANCELAM

In [ ]:
# Célula 3: Coleta e salva em CSV para cada ano e fase
# Definindo o diretório base para salvar os arquivos brutos (raw)
# Ajuste conforme a estrutura que você escolheu:
# Se o notebook está em `seu_projeto_analise_empenhos/notebooks/`, e você quer salvar em `seu_projeto_analise_empenhos/data/raw/`
# então o caminho relativo é `../data/raw/`.
BASE_RAW_DIR = "../data/raw"

for ano in ANOS:
    for fase in [1, 2, 3]:
        print(f"\nIniciando coleta para {FASENOMES[fase]} no ano {ano}...")
        df = coletar_dados_por_fase(fase, ano)
        
        if not df.empty: # Verifica se o DataFrame não está vazio antes de tentar salvar
            # Cria o subdiretório para o ano dentro de data/raw/
            ano_dir = os.path.join(BASE_RAW_DIR, str(ano))
            os.makedirs(ano_dir, exist_ok=True)
            
            # Define o nome do arquivo dentro do diretório do ano
            nome_arquivo_csv = f"{FASENOMES[fase].lower()}_{ano}.csv"
            caminho_completo_arquivo = os.path.join(ano_dir, nome_arquivo_csv)
            
            df.to_csv(caminho_completo_arquivo, index=False, encoding='utf-8-sig')
            print(f"📁 CSV salvo: {caminho_completo_arquivo}")
        else:
            print(f"⚠️ Nenhum dado coletado para {FASENOMES[fase]} no ano {ano}. CSV não salvo.")


Iniciando coleta para Empenhos no ano 2023...
✅ Empenhos 2023: Página 1 com 15 registros
✅ Empenhos 2023: Página 2 com 15 registros
✅ Empenhos 2023: Página 3 com 15 registros
✅ Empenhos 2023: Página 4 com 3 registros
📁 CSV salvo: ../data/raw\2023\empenhos_2023.csv

Iniciando coleta para Liquidações no ano 2023...
✅ Liquidações 2023: Página 1 com 15 registros
✅ Liquidações 2023: Página 2 com 15 registros
✅ Liquidações 2023: Página 3 com 15 registros
✅ Liquidações 2023: Página 4 com 15 registros
✅ Liquidações 2023: Página 5 com 15 registros
✅ Liquidações 2023: Página 6 com 15 registros
✅ Liquidações 2023: Página 7 com 15 registros
✅ Liquidações 2023: Página 8 com 15 registros
✅ Liquidações 2023: Página 9 com 15 registros
✅ Liquidações 2023: Página 10 com 15 registros
✅ Liquidações 2023: Página 11 com 15 registros
✅ Liquidações 2023: Página 12 com 15 registros
✅ Liquidações 2023: Página 13 com 15 registros
✅ Liquidações 2023: Página 14 com 15 registros
✅ Liquidações 2023: Página 15 com 1